In [1]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import numpy as np
from nltk.metrics import *
from sklearn import metrics
from os import scandir, getcwd
import json


import warnings
warnings.filterwarnings("ignore")

#### Definimos donde esta el diccionario y los tweets, ademas de que experimento realizar. La variable "clasificacion" indica si se usa clasificación binaria o ternaria

In [2]:
diccionario = "senticon.es.xml"
tweets = 'tweets_preprocessed.xlsx'
path = files_path=getcwd()
file_1 = "SentiCon_1"
file_2 = "SentiCon_2"
directory = '\\TERNARIO\\'
# directory = '\\BINARIO\\'


filename_1 = path+directory+file_1
filename_2 = path+directory+file_2
nombre = 'SentiCon'

clasificacion = 3


In [3]:
import xml.etree.cElementTree as et 
parsedXML = et.parse( "senticon.es.xml" )

#### Los archivos xml se leen utilizando un parser Xml. Este diccionario esta en xml

In [4]:
dfcols = ['Sentimiento', 'Palabra', 'Categoría', 'Posicion']
dict_label = pd.DataFrame(columns=dfcols)

for layer in parsedXML.iter('layer'):
    for positive in layer.iter('positive'):
        for lemma in positive.iter('lemma'):
            word = lemma.text
            sentiment = 'pos'
            polarity = lemma.attrib.get('pol')
            position = lemma.attrib.get('pos')
           
            
            dict_label = dict_label.append(
            pd.Series([sentiment, word, float(polarity),position],index=dfcols),ignore_index=True)
            
            
            
    for positive in layer.iter('negative'):
        for lemma in positive.iter('lemma'):
            word = lemma.text
            sentiment = 'neg'
            polarity = lemma.attrib.get('pol')
            position = lemma.attrib.get('pos')
           
            
            dict_label = dict_label.append(
            pd.Series([sentiment, word, float(polarity),position],index=dfcols),ignore_index=True)
            
            
dict_label


dict_label.to_excel('senticon.xlsx')


#### Como se eliminaron los acentos del conjunto de tweets, tambien se eliminan de las palabras del diccionario

In [5]:
def preprocesamiento_dict(row):
    a,b = 'áéíóúü','aeiouu'
    trans = str.maketrans(a,b)
    replaced_links = row["Palabra"].translate(trans)  
    replaced_links = row['Palabra'].strip()
    
    return replaced_links   

#### Aplicamos el preprocesamiento al diccionario

In [ ]:
dict_label["Palabra"] = dict_label.apply(preprocesamiento_dict, axis=1)
dict_label = dict_label.drop_duplicates(subset=['Palabra'])

dict_label = dict_label.loc[~dict_label['Palabra'].str.contains('_')]

In [ ]:
df = pd.read_excel(tweets)

In [ ]:
df_label = df[df.sentiment < clasificacion]
df_clean = df_label


In [9]:
dict_label_stem = dict_label

#### Dividimos en conjuntos de prueba y entrenamiento

In [ ]:
train=df_clean.sample(frac=0.8,random_state=200)
test=df_clean.drop(train.index)


train = train[train.sentiment < clasificacion]


df_clean = train


In [11]:
df_clean = df_clean.dropna()

In [17]:
dict_label = dict_label.reset_index()

In [ ]:
dict_label = dict_label.dropna()

#### Esta función calcula la polaridad promedio de una palabra que no se encuentra en el diccionario, a partir de las polaridades de los tweets donde aparece

In [19]:
def polaridadPromedio(palabra):
    tweetsQueTienenPalabra = df_clean[df_clean['text'].str.contains(palabra)]
    #print(tweetsQueTienenPalabra)
    avg = tweetsQueTienenPalabra["sentiment"].mean()
    return avg

In [21]:
df_aux = df_clean

for id, tweet in df_aux.iterrows():
    filtered_words = [word for word in tweet.text.split() if word not in stopwords.words('spanish')]
    for word in filtered_words:
        if dict_label.loc[dict_label['Palabra'].isin([word])]['Palabra'].any():
            a=1
        else:        
            avg = polaridadPromedio(str(word))
            df2 = pd.DataFrame([[word,avg]], columns=['Palabra','Categoría'])
            dict_label = dict_label.append(df2)

In [22]:
def analisis_sentimiento(text):
    valor = 0
    filtered_words = [word for word in str(text).split() if word not in stopwords.words('spanish')]
    for word in filtered_words:
        if dict_label.loc[dict_label['Palabra'].isin([word])]['Categoría'].any():           
            try:                
                valor = valor + float(dict_label.loc[dict_label['Palabra'].isin([word])]['Categoría'])
            except:
                print("error")                
    
    return valor

#### Esta función determina la polaridad de un tweet en funcion del diccionario

In [24]:
def analisis_sentimiento(text):
    tienePalabra = False
    valor = 999999
    filtered_words = [word for word in text.split() if word not in stopwords.words('spanish')]
    for word in filtered_words:
        if dict_label.loc[dict_label['Palabra'].isin([word])]['Categoría'].any(): 
            if not tienePalabra:
                tienePalabra = True
                valor = 0
            try:                
                valor = valor + float(dict_label.loc[dict_label['Palabra'].isin([word])]['Categoría'])
            except Exception as e: 
                print(e)
                print(dict_label.loc[dict_label['Palabra'].isin([word])]['Categoría'])               
    
    return valor

#### Se calcula la polaridad del conjunto de tweets de test


In [ ]:
correcto = 0
max = 0
min = 0
ref = []
testVal= []
sentimentList = []

if(clasificacion == 2):
    #BINARIO
    for id, tweet in test.iterrows(): 
        sentimiento = analisis_sentimiento(str(tweet.text))
        sentimentList.append(sentimiento)
        if(sentimiento != 999999):
            if(sentimiento > max):
                max = sentimiento
            if(sentimiento < min):
                min = sentimiento  
            ref.append(1 if int(tweet.sentiment) == 1 else -1)      
            if(sentimiento <= 3): 
                testVal.append(-1)
            else:
                testVal.append(1)
        
else:
    #TERNARIO
    for id, tweet in test.iterrows():    
        sentimiento = analisis_sentimiento(str(tweet.text))        
        if (int(tweet.sentiment) == 0):
            valor_tweet = -1
        else: 
            valor_tweet = int(tweet.sentiment)
        if(sentimiento != 999999):
            if(sentimiento > max):
                max = sentimiento
            if(sentimiento < min):
                min = sentimiento
            ref.append(valor_tweet) 
            sentimentList.append(sentimiento)
            if sentimiento > 8 and sentimiento < 13:
                testVal.append(2)
            elif(sentimiento <= 8): 
                testVal.append(-1)
            else:
                testVal.append(1)

        

        

#### Guardamos los resultados de la clasificación.

In [28]:

file1 = open(filename_1 + '.txt',"w") 

accuracyDict = {}
  
# \n is placed to indicate EOL (End of Line) 
file1.write("Resultados "+ filename_1 + ' \n\n') 

writer = pd.ExcelWriter(path+directory+'clasification_report_'+file_1+'.xlsx', engine='xlsxwriter')
writerCM = pd.ExcelWriter(path+directory+'confusion_matrix_'+file_1+'.xlsx', engine='xlsxwriter')

#     print("***** METRICAS CLASIFICADOR "+nombre+" *****")
file1.write("***** METRICAS DICCIONARIO: SentiCon *****\n\n") 
#     print("Exactitud (accuracy) " +  str(accuracy(ref, test)*100))
file1.write("Exactitud (accuracy) " +  str(accuracy(ref, testVal)*100)+"\n") 
reference_set = set(ref)
test_set = set(testVal)
#     print("Precision " +str(precision(reference_set, test_set)))
file1.write("Precision " +str(precision(reference_set, test_set))+"\n")
#     print("Recall " +str(recall(reference_set, test_set)))
file1.write("Recall " +str(recall(reference_set, test_set))+"\n")
#     print("F measure " + str(f_measure(reference_set, test_set)))
file1.write("F measure " + str(f_measure(reference_set, test_set))+"\n\n")
file1.write("Matriz de confusión \n")
#     print(ConfusionMatrix(ref, test))   
file1.write(str(ConfusionMatrix(ref, testVal)))
file1.write('\n')
file1.write("Reporte de clasificación\n")
#     print(metrics.classification_report(ref,test,digits=3))
file1.write(metrics.classification_report(ref,testVal,digits=3)+'\n')
    
    
    
data = {'y_Predicted': testVal,
    'y_Actual':   ref
    }

df_cm = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df_cm['y_Actual'], df_cm['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])

confusion_matrix.to_excel(writerCM,sheet_name=nombre) 

report = metrics.classification_report(ref, testVal, output_dict=True)
df = pd.DataFrame(report).transpose()
df.to_excel(writer,sheet_name=nombre)


file1.write("****************************************\n\n\n\n")

accuracyDict[nombre] = str(accuracy(ref, testVal)*100)

df_ac = pd.DataFrame([accuracyDict])   
df_ac = df_ac.T
df_ac.to_excel(path+directory+'accuracy_'+file_1+'.xlsx')

file1.close() #to change file access modes 
writer.save()


# Stemmer


#### Aplicamos stemming a la clasificación con diccionarios.

In [29]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

#### Función que aplica el algoritmo de stemming a un texto.


In [30]:
def lematizacion(row):
    result = stemmer.stem(row['Palabra'])
    return result

In [31]:
dict_label= dict_label_stem
dict_label['Palabra'] = dict_label.apply(lematizacion, axis=1)
dict_label

,Sentimiento,Palabra,Categoría,Posicion
0,pos,acert,0.708,a
1,pos,admir,0.906,a
2,pos,admir,0.450,n
3,pos,admir,0.750,v
4,pos,afect,0.375,a
5,pos,afect,0.321,n
6,pos,afectu,0.563,a
7,pos,aficion,0.500,n
8,pos,afortun,0.813,a
9,pos,agrad,0.750,a


#### Elimnamos las palabras duplicadas que puedan estar en el diccionario

In [32]:
dict_label = dict_label.drop_duplicates(subset=['Palabra'])
dict_label

,Sentimiento,Palabra,Categoría,Posicion
0,pos,acert,0.708,a
1,pos,admir,0.906,a
4,pos,afect,0.375,a
6,pos,afectu,0.563,a
7,pos,aficion,0.500,n
8,pos,afortun,0.813,a
9,pos,agrad,0.750,a
10,pos,agradec,0.500,a
11,pos,alegr,0.458,v
13,pos,alent,0.594,a


#### Esta función determina la polaridad de un tweet en funcion del diccionario

In [33]:
#CON STEMMING
def analisis_sentimiento(text):
    tienePalabra = False
    valor = 999999
    filtered_words = [word for word in text.split() if word not in stopwords.words('spanish')]
    for word in filtered_words:
        if dict_label.loc[dict_label['Palabra'].isin([stemmer.stem(word)])]['Categoría'].any(): 
            if not tienePalabra:
                tienePalabra = True
                valor = 0
            try:                
                valor = valor + float(dict_label.loc[dict_label['Palabra'].isin([stemmer.stem(word)])]['Categoría'])
            except Exception as e: 
                print(e)
                print(dict_label.loc[dict_label['Palabra'].isin([word])]['Categoría'])               
    
    return valor

#### Se calcula la polaridad del conjunto de tweets de test


In [ ]:
correcto = 0
max = 0
min = 0
ref = []
testVal= []

if(clasificacion == 2):
    #BINARIO
    for id, tweet in test.iterrows(): 
        sentimiento = analisis_sentimiento(str(tweet.text))
        if(sentimiento != 999999):
            if(sentimiento > max):
                max = sentimiento
            if(sentimiento < min):
                min = sentimiento  
            ref.append(1 if int(tweet.sentiment) == 1 else -1)      
            if(sentimiento <= 1): 
                testVal.append(-1)
            else:
                testVal.append(1)
        
else:
    #TERNARIO
    for id, tweet in test.iterrows():    
        sentimiento = analisis_sentimiento(str(tweet.text))
        if (int(tweet.sentiment) == 0):
            valor_tweet = -1
        else: 
            valor_tweet = int(tweet.sentiment)
        if(sentimiento != 999999):
            if(sentimiento > max):
                max = sentimiento
            if(sentimiento < min):
                min = sentimiento
            ref.append(valor_tweet)  
            if sentimiento > 0.5 and sentimiento < 1.5:
                testVal.append(2)
            elif(sentimiento <= 0): 
                testVal.append(-1)
            else:
                testVal.append(1)

        



#### Guardamos los resultados de la clasificación.

In [37]:

file1 = open(filename_2 + '.txt',"w") 

accuracyDict = {}
  
# \n is placed to indicate EOL (End of Line) 
file1.write("Resultados "+ filename_2 + ' \n\n') 

writer = pd.ExcelWriter(path+directory+'clasification_report_'+file_2+'_STEMMING.xlsx', engine='xlsxwriter')
writerCM = pd.ExcelWriter(path+directory+'confusion_matrix_'+file_2+'_STEMMING.xlsx', engine='xlsxwriter')

#     print("***** METRICAS CLASIFICADOR "+nombre+" *****")
file1.write("***** METRICAS DICCIONARIO: SentiCon *****\n\n") 
#     print("Exactitud (accuracy) " +  str(accuracy(ref, test)*100))
file1.write("Exactitud (accuracy) " +  str(accuracy(ref, testVal)*100)+"\n") 
reference_set = set(ref)
test_set = set(testVal)
#     print("Precision " +str(precision(reference_set, test_set)))
file1.write("Precision " +str(precision(reference_set, test_set))+"\n")
#     print("Recall " +str(recall(reference_set, test_set)))
file1.write("Recall " +str(recall(reference_set, test_set))+"\n")
#     print("F measure " + str(f_measure(reference_set, test_set)))
file1.write("F measure " + str(f_measure(reference_set, test_set))+"\n\n")
file1.write("Matriz de confusión \n")
#     print(ConfusionMatrix(ref, test))   
file1.write(str(ConfusionMatrix(ref, testVal)))
file1.write('\n')
file1.write("Reporte de clasificación\n")
#     print(metrics.classification_report(ref,test,digits=3))
file1.write(metrics.classification_report(ref,testVal,digits=3)+'\n')
    
    
    
data = {'y_Predicted': testVal,
    'y_Actual':   ref
    }

df_cm = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df_cm['y_Actual'], df_cm['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])

confusion_matrix.to_excel(writerCM,sheet_name=nombre) 

report = metrics.classification_report(ref, testVal, output_dict=True)
df = pd.DataFrame(report).transpose()
df.to_excel(writer,sheet_name=nombre)


file1.write("****************************************\n\n\n\n")

accuracyDict[nombre] = str(accuracy(ref, testVal)*100)

df_ac = pd.DataFrame([accuracyDict])   
df_ac = df_ac.T
df_ac.to_excel(path+directory+'accuracy_'+file_2+'_STEMMING.xlsx')

file1.close() #to change file access modes 
writer.save()
